# Prograd

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'..\databases\Classificação Geral do Ingresso\bd_prograd01_2024.csv', sep=';', encoding='latin-1')


## EDA

In [10]:
df.columns

Index(['inscrição no ENEM', 'nome candidato', 'opção de curso',
       'opção de campus', 'opção de turno', 'modalidade inscrição', 'nota',
       'classificação original', 'classificação atual', 'status de matrícula',
       'chamada', 'modalidade convocação'],
      dtype='object')

In [11]:
# Dimensões do DataFrame
print(f"Dimensões do DataFrame: {df.shape}")

# Informações gerais
df.info()

# Verificar valores nulos
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# Estatísticas descritivas para colunas numéricas
print("\nEstatísticas descritivas:")
df.describe()

Dimensões do DataFrame: (9659, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   inscrição no ENEM       9659 non-null   object
 1   nome candidato          9659 non-null   object
 2   opção de curso          9659 non-null   object
 3   opção de campus         9659 non-null   object
 4   opção de turno          9659 non-null   object
 5   modalidade inscrição    9659 non-null   object
 6   nota                    9659 non-null   object
 7   classificação original  9659 non-null   int64 
 8   classificação atual     9659 non-null   int64 
 9   status de matrícula     9659 non-null   object
 10  chamada                 2949 non-null   object
 11  modalidade convocação   2949 non-null   object
dtypes: int64(2), object(10)
memory usage: 905.7+ KB

Valores nulos por coluna:
inscrição no ENEM            0
nome candidato     

,classificação original,classificação atual
count,9659.000000,9659.000000
mean,174.626773,74.097836
std,192.336839,132.140263
min,1.000000,0.000000
25%,32.000000,0.000000
50%,99.000000,0.000000
75%,260.000000,90.000000
max,905.000000,669.000000


### 1. Análise da modalidade de convocação

In [30]:
print(df.chamada.value_counts(dropna=False), '\n')
print(df['modalidade convocação'].value_counts(dropna=False))

chamada
NaN                   6710
Chamada Regular       1845
1ª Lista de Espera     610
2ª Lista de Espera     221
4ª Lista de Espera     164
3ª Lista de Espera     109
Name: count, dtype: int64 

modalidade convocação
NaN                             6710
Ampla concorrência              1293
Escola Pública + Renda           417
PPI + Renda                      393
Escola Pública                   343
PPI                              303
Escola Pública + PCD              47
Escola Pública + PCD + Renda      47
PCD                               45
Pessoa Transgênera                23
Pessoa Transgênera + Renda        15
Quilombola                        11
Quilombola + Renda                 9
Refugiado                          2
Refugiado + Renda                  1
Name: count, dtype: int64


Os candidatos podem estar em mais de uma modadlidade de inscrição, mas convocados em apenas uma delas.
O campo modalidade de convocação é preenchido quando o candidato é convocado, e pode ser usado para unificar candidatos que estão em mais de uma modalidade de inscrição.	

Classificação original

In [49]:
df.loc[df['classificação atual'] == 1].head()
# print(df['chamada'].unique())
# df.loc[df['chamada'].isnull()]
# df.loc[df['nome candidato'].str.contains('MARINA LIMA')]

,inscrição no ENEM,nome candidato,opção de curso,opção de campus,opção de turno,modalidade inscrição,nota,classificação original,classificação atual,status de matrícula,chamada,modalidade convocação
380,23****382708,MARIA EDUARDA FERNANDES HORTA RODRIGUES,BACHARELADO EM CIÊNCIA E TECNOLOGIA,SANTO ANDRÉ,MATUTINO,Ampla concorrência,"671,20",183,1,Não Convocado(a),NaN,NaN
1088,23****157391,PEDRO HENRIQUE CAMPOS RIBEIRO,BACHARELADO EM CIÊNCIA E TECNOLOGIA,SANTO ANDRÉ,MATUTINO,Escola Pública,"670,95",33,1,Não Convocado(a),NaN,NaN
1634,23****386073,NICOLAS PEREIRA SILVA SANTOS,BACHARELADO EM CIÊNCIA E TECNOLOGIA,SANTO ANDRÉ,MATUTINO,Escola Pública + Renda,"612,61",52,1,Não Convocado(a),NaN,NaN
1855,23****017942,ERICA SANTOS AVEDO,BACHARELADO EM CIÊNCIA E TECNOLOGIA,SANTO ANDRÉ,MATUTINO,PPI,"628,36",31,1,Não Convocado(a),NaN,NaN
2094,23****546264,MATHEUS FERNANDES DE CARVALHO,BACHARELADO EM CIÊNCIA E TECNOLOGIA,SANTO ANDRÉ,MATUTINO,PPI + Renda,"565,56",46,1,Não Convocado(a),NaN,NaN


# MEC

In [1]:
from pyspark.sql import SparkSession

# Iniciar uma sessão Spark
spark = SparkSession.builder \
    .appName("Análise SISU") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Ler o CSV com Spark
df_mec = spark.read.format("csv") \
    .option("header", "true") \
    .option("sep", "|") \
    .option("encoding", "latin1") \
    .load(r"..\databases\MEC\chamada_regular_sisu_2023_1.csv")

# Mostrar as primeiras linhas para verificar
df_mec.show(5)

+----+------+-----+---------------+----------+--------------------+---------+------+-------------+--------------------+---------+----------------+------------+--------------------+-----------+-------+----------------+-------+---------------------+------------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+-------------+--------------+--------------------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+---------+
| ANO|EDICAO|ETAPA|       DS_ETAPA|CODIGO_IES|            NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONCORRENCI

In [5]:
spark.stop()